This pipeline is designed to train the model in an automated way.
First, we will define a config with all the specifications. This dictionary is vital to keep track of the models we have used.
Then run the model and extract the metrics that we need, to evaluate the training.

In [1]:
#import config from file
import json

with open("config.json", 'r') as f:
    config = json.load(f)

print(config)

{'model': 'sequence', 'submodel': 'implicit', 'loss': 'pointwise', 'representation': 'pooling', 'embedding_dim': 32, 'n_iter': 10, 'batch_size': 256, 'l2': 0.0, 'lr': 0.01, 'optim': None, 'use_cuda': False, 'sparse': False, 'random_state': None, 'num_negative_samples': 5, 'dataset': 'Movielens', 'size': '100K'}


In [2]:
# or define it manually

# for sequence
config = {
    # About the model
    "model": "sequence",    # sequence or factorizer
    "submodel": "implicit",    # implicit or explicit
    "loss": "bpr",    # one of ‘pointwise’, ‘bpr’, ‘hinge’, ‘adaptive_hinge’ or 'regression', 'poisson','logistic'
    "representation": "pooling",    # for sequence one of ‘pooling’, ‘cnn’, ‘lstm’, ‘mixture’, for factorizer always NONE !!!
    "embedding_dim": 32, 
    "n_iter": 10,
    "batch_size": 256,
    "l2": 0.0,
    "lr": 0.01,
    "optim": None,
    "use_cuda": False, 
    "sparse": False, 
    "random_state": None, 
    "num_negative_samples": 5,

    # About the database
    "dataset": "Movielens",    # Movielens, Synthetic or Goodbooks
    "size": "100K",
    # synthetic has a lot of different parameters but I'm not sure we are going to use it
}

In [12]:
# for factorizer
config = {
    # About the model
    "model": "factorizer",    # sequence or factorizer
    "submodel": "implicit",    # implicit or explicit
    "loss": "bpr",    # one of ‘pointwise’, ‘bpr’, ‘hinge’, ‘adaptive_hinge’ or 'regression', 'poisson','logistic'
    "representation": None,    # for sequence one of ‘pooling’, ‘cnn’, ‘lstm’, ‘mixture’, for factorizer always None !!!
    "embedding_dim": 32, 
    "n_iter": 10,
    "batch_size": 256,
    "l2": 0.0,
    "lr": 0.01,
    "optim": None,
    "use_cuda": False, 
    "sparse": False, 
    "random_state": None, 
    "num_negative_samples": 5,

    # About the database
    "dataset": "Movielens",    # Movielens, Synthetic or Goodbooks
    "size": "100K",
    # synthetic has a lot of different parameters but I'm not sure we are going to use it
}

In [3]:
from settings_definition import *

In [4]:
def evaluate_model(model, test):
    """This function evaluates the metric agreed by the team.
    At the moment we have not decided which ones so I just pick 2 as an example."""
    from spotlight.evaluation import mrr_score, precision_recall_score, rmse_score, sequence_mrr_score, sequence_precision_recall_score
    eval = {"mrr": mrr_score(model, test), "precision_recall": precision_recall_score(model, test), #"rmse": rmse_score(model, test),
            #"sequence_mrr":sequence_mrr_score(model, test), "sequence_precision_recall": sequence_precision_recall_score(model, test)
           }
    return eval

In [5]:
import os
os.chdir("../spotlight")
# for me, it only works if I'm in the spotlight directory

In [13]:
# Execution

from spotlight.cross_validation import random_train_test_split

dataset = define_dataset(config)
train, test = random_train_test_split(dataset)
if config["model"] == "sequence":
    train, test = train.to_sequence(), test
model = define_model(config)
model.fit(train)

eval = evaluate_model(model, test)
eval

{'mrr': array([0.01522165, 0.01620213, 0.05142308, 0.0844071 , 0.0093615 ,
        0.02616351, 0.00817803, 0.01525699, 0.00222461, 0.03499661,
        0.02417787, 0.03347241, 0.01025752, 0.03415652, 0.01185052,
        0.07126002, 0.16205211, 0.03708636, 0.02614923, 0.04634411,
        0.00938333, 0.02391037, 0.06447179, 0.03485353, 0.05184816,
        0.02537377, 0.04943088, 0.06775143, 0.02066926, 0.15546364,
        0.07338927, 0.02934762, 0.03924703, 0.02329678, 0.15034409,
        0.03608729, 0.01968232, 0.02246735, 0.03483288, 0.04239267,
        0.05999913, 0.0216327 , 0.0124676 , 0.02943506, 0.02124338,
        0.03385237, 0.52631579, 0.02402827, 0.00621224, 0.08181818,
        0.05770903, 0.09246824, 0.07725337, 0.08244367, 0.11521752,
        0.0294225 , 0.02678522, 0.04007842, 0.0174609 , 0.03516826,
        0.05803026, 0.03928367, 0.11131422, 0.01912906, 0.03713121,
        0.17051306, 0.13574929, 0.05805326, 0.02507548, 0.01420426,
        0.20366751, 0.02044519, 0.0683645

In [14]:
import torch

torch.save(model, "../pipeline/trained_model_" + config["model"])
print("saved to " + "../pipeline/trained_model_" + config["model"])

saved to ../pipeline/trained_model_factorizer
